<a href="https://colab.research.google.com/github/GuilhermeCaly/fraud_credit/blob/main/fraud_credit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
import pandas as pd
import numpy as np
from datetime import datetime
import pymysql
from google.colab import userdata

In [9]:
host = userdata.get('host')
user = userdata.get('user')
password = userdata.get('password')
database = userdata.get('database')
drive.mount('/content/drive')
file_path = userdata.get('file_path')
dt_source_lc = pd.read_csv(file_path)
df_fraud =  dt_source_lc

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
df_fraud.dropna(inplace=True)
df_fraud = df_fraud[(df_fraud['location_region'] != '0') & (df_fraud['amount'] != "none") & (df_fraud['risk_score'] != "none")]
df_fraud = df_fraud[(df_fraud['amount'] != "none") & (df_fraud['risk_score'] != "none")]
df_fraud['timestamp'] = pd.to_datetime(df_fraud['timestamp'], unit='s')

In [11]:
df_fraud['sending_address'] =  df_fraud['sending_address'].astype(str)
df_fraud['risk_score'] =  df_fraud['risk_score'].astype(float)
df_fraud['location_region'] =  df_fraud['location_region'].astype(str)
df_fraud['receiving_address'] =  df_fraud['receiving_address'].astype(str)
df_fraud['amount'] =  df_fraud['amount'].astype(float)
df_fraud['transaction_type'] =  df_fraud['transaction_type'].astype(str)
df_fraud['ip_prefix'] =  df_fraud['ip_prefix'].astype(str)
df_fraud['login_frequency'] =  df_fraud['login_frequency'].astype(int)
df_fraud['session_duration'] =  df_fraud['session_duration'].astype(int)
df_fraud['purchase_pattern'] =  df_fraud['purchase_pattern'].astype(str)
df_fraud['age_group'] =  df_fraud['age_group'].astype(str)
df_fraud['anomaly'] =  df_fraud['anomaly'].astype(str)

In [12]:
mean_risk_score = df_fraud.groupby('location_region')['risk_score'].mean()
result_table_1 = mean_risk_score.sort_values(ascending=False).reset_index()
display(result_table_1)

,location_region,risk_score
0,North America,45.154834
1,South America,45.139408
2,Asia,44.994572
3,Africa,44.902219
4,Europe,44.598708


In [13]:
sales_data = df_fraud[df_fraud['transaction_type'] == 'sale']
latest_transactions = sales_data.loc[sales_data.groupby('receiving_address')['timestamp'].idxmax()]
result_table_2 = latest_transactions.nlargest(3, 'amount')[['receiving_address', 'amount', 'timestamp']]
result_table_2

,receiving_address,amount,timestamp
2927905,0x841342e50c508ec4ffdef9b5208719c1dbed7968,76568.0,2024-01-02 03:53:01
5871321,0xe8aacdea4f2d7658e711de611bad8e3b5d6b2c7b,76563.0,2024-01-01 02:49:56
1140689,0x231dd8e2959e878a59a26ebdbf6f7d122403f350,76559.0,2024-01-02 06:31:09


In [40]:
log_error = dt_source_lc.copy()
log_error = dt_source_lc[(dt_source_lc['location_region'] == '0') |
                         (dt_source_lc['amount'] == "none") |
                         (dt_source_lc['risk_score'] == "none")]

In [41]:
log_error['error_message'] = ''
log_error.loc[log_error['location_region'] == '0', 'error_message'] = 'REGION DOES NOT EXISTS'
log_error.loc[log_error['amount'] == "none", 'error_message'] = 'AMOUNT IS NONE'
log_error.loc[log_error['risk_score'] == "none", 'error_message'] = 'RISK SCORE IS NONE'

<ipython-input-41-b689ed2e23d2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  log_error['error_message'] = ''


In [16]:
quantidade_erros_log = len(log_error)
erros_region = log_error.loc[log_error['error_message'] == 'REGION DOES NOT EXISTS']
erros_region_all = len(erros_region)
erros_amount = log_error.loc[log_error['error_message'] == 'AMOUNT IS NONE']
erros_amount = len(erros_amount)
erros_risk_score = log_error.loc[log_error['error_message'] == 'RISK SCORE IS NONE']
erros_risk_score = len(erros_risk_score)
quantidade_total_registros = len(dt_source_lc)
percentual_conformidade_log = 100 * (quantidade_total_registros - quantidade_erros_log) / quantidade_total_registros


In [17]:
last_insert_date = datetime.now()

data_quality= {
    'qtd_erros_Log': [quantidade_erros_log],
    'qtd_erros_region__not_exits': [erros_region_all],
    'qtd__erros_amount_none': [erros_amount],
    'qtd_erros_risk_none': [erros_risk_score],
    'qtd_total_registros': [quantidade_total_registros],
    'percent_conf_log': [percentual_conformidade_log],
    'Dt_ultimo_inset': [last_insert_date]
}
data_quality = pd.DataFrame(data_quality)

In [18]:
data_quality = pd.DataFrame(data_quality)

In [19]:
data_quality

,qtd_erros_Log,qtd_erros_region__not_exits,qtd__erros_amount_none,qtd_erros_risk_none,qtd_total_registros,percent_conf_log,Dt_ultimo_inset
0,149406,49456,49724,50226,9291894,98.392082,2024-04-15 12:03:13.151155


In [20]:
conn = pymysql.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

In [21]:
conn.open
cursor = conn.cursor()

In [26]:
for index, row in data_quality.iterrows():
    insert_tbl_dt_qlt = "INSERT INTO DB_FRAUD.DT_QTLT (qtd_erros_log, qtd_erros_region__not_exits, qtd__erros_amount_none, qtd_erros_risk_none, qtd_total_registros, percent_conf_log, Dt_ultimo_inset) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    data_quality_values = (row['qtd_erros_Log'], row['qtd_erros_region__not_exits'], row['qtd__erros_amount_none'], row['qtd_erros_risk_none'], row['qtd_total_registros'], row['percent_conf_log'], row['Dt_ultimo_inset'])
    cursor.execute(insert_tbl_dt_qlt, data_quality_values)
    conn.commit()

In [ ]:
for index , row in log_error.iterrows():
    insert_logerror = "INSERT INTO DB_FRAUD.DT_LOGERROR(`timestamp` ,`sending_address`  ,`receiving_address`    ,`amount`   ,`transaction_type` ,`location_region`  ,`ip_prefix`    ,`login_frequency`  ,`session_duration` ,`purchase_pattern` ,`age_group`    ,`risk_score`   ,`anomaly`  ,`error_message`)  VALUES (%s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s) "
    log_error_insert = (row['timestamp'],row['sending_address'],row['receiving_address'],row['amount'],row['transaction_type'],row['location_region'],row['ip_prefix'],row['login_frequency'],row['session_duration'],row['purchase_pattern'],row['age_group'],row['risk_score'],row['anomaly'],row['error_message'],)
    cursor.execute(insert_logerror, log_error_insert)
    conn.commit()

In [ ]:
    cursor.close()
    conn.close()
    print("Conexão ao MySQL fechada.")

In [49]:
log_error

,timestamp,sending_address,receiving_address,amount,transaction_type,location_region,ip_prefix,login_frequency,session_duration,purchase_pattern,age_group,risk_score,anomaly,error_message
12,1590044893,0xfcf9a3467d3d93688fb17b4ebb6b681b1fa29f94,0x6d4f7db757a1bd6bb43029ee13c037651ca20c74,65319.0,purchase,North America,172.160,4,62,focused,established,none,low_risk,RISK SCORE IS NONE
13,1661550384,0xd2cff98e8e707049db92500414fec6f0bb5c895c,0x2e0925b922fed01f6a85d213ae2718f54b8ca305,67174.0,purchase,South America,192.000,2,38,random,new,none,low_risk,RISK SCORE IS NONE
21,1670045880,0xddbe1291b454f9b8699220f1e8724aa641ef4b42,0x51c0d24ccc5d9b0dd793052cb2d41efde2568056,15462.0,sale,Africa,172.000,6,144,high_value,veteran,none,low_risk,RISK SCORE IS NONE
81,1669402947,0xffa670245089044b1e355508a7843692a25a5e52,0xdce371e2762ac4250469f709c5b766e754e2c9a2,59980.0,purchase,Asia,10.000,1,38,random,new,none,low_risk,RISK SCORE IS NONE
182,1606819329,0xc3b34de3e302dc5c357c9647b1bd223eea847ff5,0xd08041262c12eb441ffb76d90aac6bbd4ead64f9,19323.0,transfer,0,172.160,8,99,high_value,veteran,36.75,low_risk,REGION DOES NOT EXISTS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9291732,1697832974,0x0ce96e253ce21998420a9e9797a4b6acce9c5ad0,0x8e4ac7b757d4d65821a4cd56405544b6aa82de94,18273.0,purchase,0,172.160,2,20,random,new,49.6125,low_risk,REGION DOES NOT EXISTS
9291747,1580608141,0x873b018df74633a9c5cd57af9e745c1e709630f8,0xda6f58d5369d83a6d4307e43b822d1b01cda0a82,26673.0,scam,0,192.000,1,25,random,new,94.5,high_risk,REGION DOES NOT EXISTS
9291823,1603236791,0x327e678f2948de6540db98a7aa44d56f3e81c007,0xe0e56a2db04ea53331dc77010b28bc94280c37e5,none,purchase,Europe,192.168,3,51,focused,established,42.1875,low_risk,AMOUNT IS NONE
9291863,1617028732,0xe318bebbc290f29e4a44e21b6d4b066ccb29f573,0x1fa2e5508e01fa9c567f9762e8eba4d7848642d5,56229.0,sale,0,192.000,1,35,random,new,42.0,low_risk,REGION DOES NOT EXISTS
